# RL & MDP Project: LunarLander


In [ ]:
# If needed, install dependencies (uncomment as appropriate)
# !pip install "gymnasium[box2d]" imageio

import numpy as np
import gymnasium as gym
import imageio
from base64 import b64encode
from IPython.display import HTML

ENV_IDS = ["LunarLander-v3", "LunarLander-v2"] 


def make_lander_env(render_mode=None):
    last_error = None
    for env_id in ENV_IDS:
        try:
            if render_mode is None:
                env = gym.make(env_id)
            else:
                env = gym.make(env_id, render_mode=render_mode)
            print(f"Using env_id={env_id!r}, render_mode={render_mode!r}")
            return env
        except Exception as e:
            last_error = e
    raise RuntimeError(f"Could not create any LunarLander environment from {ENV_IDS}: {last_error}")


def record_video(env, policy, out_directory, fps=30, random_action=False, max_steps=1600, action_set=None):
    images = []
    total_reward = 0.0
    terminated = False
    truncated = False
    obs, info = env.reset()
    img = env.render()
    images.append(img)
    t = 0
    while not (terminated or truncated):
        t += 1
        if t > max_steps:
            break
        
        if random_action:
            if action_set is not None:
                idx = np.random.randint(len(action_set))
                action = np.array(action_set[idx], dtype=np.float32)
            else:
                action = env.action_space.sample()
        else:
            act = policy(obs)
            if action_set is not None:
                idx = int(act)
                action = np.array(action_set[idx], dtype=np.float32)
            else:
                if hasattr(env.action_space, "n"):
                    action = int(act)
                else:
                    action = np.array(act, dtype=np.float32)
        
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += float(reward)
        img = env.render()
        images.append(img)
    
    imageio.mimsave(out_directory, images, fps=fps)
    return total_reward


def show_video(video_path, video_width=500):
    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f'<video width="{video_width}" controls>'
                f'<source src="{video_url}" type="video/mp4"></video>')

## 1. LunarLander setup and random baseline

In this section you will:

1. Create a LunarLander environment from Gymnasium.
2. Inspect observation and action spaces.
3. Run a simple random policy baseline.
4. Record a short video of a random agent using the provided helpers.


In [ ]:
# TODO 1.1: Create a training environment (no render_mode)
train_env = None

# TODO 1.2: Create a separate environment for video recording (render_mode="rgb_array")
video_env = None

# TODO 1.3: Reset training env and inspect spaces
obs, info = None, None

print("Initial observation:", obs)
print("Observation shape:", ...)
print("Action space:", ...)

In [ ]:
# TODO === 1.2 Random policy baseline ===
def run_random_policy(env, num_episodes, max_steps):
    returns = []
    for ep in range(num_episodes):
        # TODO 1.4: reset the environment
        obs, info = None, None  # TODO
        
        total_reward = 0.0
        for t in range(max_steps):
            # TODO 1.5: sample a random action from env.action_space
            action = None
            
            # TODO 1.6: step the environment and accumulate reward
        returns.append(total_reward)
    return returns

random_returns = run_random_policy(train_env, num_episodes=..., max_steps=...)
print("Random policy: mean return =", np.mean(random_returns), "±", np.std(random_returns))

In [ ]:
# TODO === 1.3 Record a random policy video ===

## 2. A small finite MDP

To practice Iterative Policy Evaluation and Policy Iteration, we will use a small tabular MDP

In [ ]:
def build_gridworld_mdp():
    # TODO 2.0: Implement the gridworld MDP construction
    n_rows, n_cols = ..., ...
    n_states = n_rows * n_cols
    n_actions = ...
    
    # TODO: Initialize P and R arrays with appropriate shapes and dtype
    P = None
    R = None
    
    # TODO: Define terminal states (top-left corner and bottom-right corner)
    terminal_states = []  # TODO
    
    # TODO: Helper function to convert (row, col) to state index
    def to_state(r, c):
        # TODO: implement conversion from grid coordinates to state index
        pass
    
    # TODO: Fill in transition probabilities and rewards    
    return n_states, n_actions, P, R, terminal_states


n_states, n_actions, P, R, terminal_states = build_gridworld_mdp()
print("Gridworld: n_states =", n_states, "| n_actions =", n_actions)
print("Terminal states:", terminal_states)

### 2.1 Policy representation and Iterative Policy Evaluation

We represent a policy $\pi(a \mid s)$ as a 2D NumPy array of shape `[n_states, n_actions]`,
where each row is a probability distribution over actions in that state.

**Tasks:**

1. Define an **initial policy** that is **uniform over all actions** in non-terminal states.
2. Implement **Iterative Policy Evaluation**:

   $$V_{k+1}(s) = \sum_a \pi(a\mid s) \sum_{s'} P(s' \mid s, a) \left[R(s,a,s') + \gamma V_k(s') \right]$$

   Stop when the maximum change `delta` over all states is below a small threshold `theta`.

In [ ]:
gamma_grid = ...

# TODO 2.1: Define an initial uniform policy over actions (for non-terminal states)
policy = None

def iterative_policy_evaluation(policy, P, R, gamma=..., theta=...):
    n_states, n_actions, _ = P.shape
    V = np.zeros(n_states, dtype=np.float32)
    # TODO 2.2: implement the iterative policy evaluation loop

# TODO: After implementing both policy and iterative_policy_evaluation, uncomment:
V_pi = iterative_policy_evaluation(policy, P, R, gamma=gamma_grid, theta=...)
print("Value function under uniform policy (reshaped 4x4):")
print(V_pi.reshape(4, 4))

### 2.2 Policy Improvement and Policy Iteration

We now implement:

1. **Policy Improvement**: given $V(s)$, compute a greedy policy:
   $$\pi_{\text{greedy}}(s) = \arg\max_a \sum_{s'} P(s' \mid s,a) \left[ R(s,a,s') + \gamma V(s') \right]$$

2. **Policy Iteration**: alternate between policy evaluation and policy improvement until the policy stops changing.

In [ ]:
def policy_improvement(V, P, R, gamma=...):
    n_states, n_actions, _ = P.shape
    new_policy = np.zeros((n_states, n_actions), dtype=np.float32)

    # TODO 2.3: implement greedy improvement step

def policy_iteration(P, R, gamma=..., theta=..., max_iterations=...):
    n_states, n_actions, _ = P.shape

    # TODO 2.4: implement policy_iteration

# TODO: After implementing policy_improvement and policy_iteration, uncomment:
policy_star, V_star = policy_iteration(P, R, gamma=gamma_grid, theta=...)
print("Optimal value function (4x4):")
print(V_star.reshape(4, 4))
print("Optimal policy (action indices, 4x4):")
print(np.argmax(policy_star, axis=1).reshape(4, 4))

## 3. Tabular Q-Learning on LunarLander (with discretized state)

LunarLander has a **continuous state space**, so we cannot directly store a Q-table indexed by states.

We will:

1. **Discretize the 8-dimensional state vector** into a finite number of bins per dimension.
2. Maintain a **Q-table** `Q[discrete_state, action]`.
3. Implement **tabular Q-Learning**:

   $$Q(s,a) \leftarrow (1-\alpha) Q(s,a) + \alpha \left[ r + \gamma \max_{a'} Q(s',a') \right]$$

4. Train a policy and compare its returns to the random baseline.

In [ ]:
obs_low = train_env.observation_space.low
obs_high = train_env.observation_space.high
print("Observation low:", obs_low)
print("Observation high:", obs_high)

n_actions_lander = train_env.action_space.n
print("Number of discrete actions:", n_actions_lander)

num_bins = np.array([6, 6, 6, 6, 6, 6, 2, 2], dtype=int)

# TODO 3.1: Create bin boundaries for each dimension
def create_bins(obs_low, obs_high, num_bins):
    raise NotImplementedError("TODO 3.1: implement create_bins")


bins = None

# TODO 3.2: Discretize a continuous observation into a tuple of indices
def discretize_observation(obs, bins):
    raise NotImplementedError("TODO 3.2: implement discretize_observation")

test_state = discretize_observation(obs, bins)
print("Example discrete state:", test_state)

In [ ]:
alpha = ...
gamma_q = ...
epsilon_start = ...
epsilon_end = ...
epsilon_decay = ...
num_episodes = ...
max_steps_per_episode = ...

state_shape = tuple(num_bins)
q_table = None


# TODO 3.3: epsilon-greedy action selection
def epsilon_greedy_action(q_table, state, epsilon, n_actions):
    raise NotImplementedError("TODO 3.3: implement epsilon-greedy action selection")


# TODO 3.4: Q-Learning training loop
def q_learning_train(env, bins, num_bins, num_episodes=..., max_steps=...,
                     alpha=..., gamma=...,
                     epsilon_start=..., epsilon_end=..., epsilon_decay=...):
    raise NotImplementedError("TODO 3.4: implement Q-Learning training loop")


q_table, episode_returns = q_learning_train(
    train_env, bins, num_bins,
    num_episodes=num_episodes,
    max_steps=max_steps_per_episode,
    alpha=alpha,
    gamma=gamma_q,
    epsilon_start=epsilon_start,
    epsilon_end=epsilon_end,
    epsilon_decay=epsilon_decay,
)

In [ ]:
import matplotlib.pyplot as plt
# TODO 3.5: Plot learning curve once you have episode_returns

In [ ]:
# TODO 3.6: deterministic greedy policy from q_table
def q_table_greedy_policy(q_table, bins):
    raise NotImplementedError("TODO 3.6: implement greedy policy from q_table")

In [ ]:
# TODO === 3.7 Record a random policy video ===